In [6]:
! pip install nlpsandbox-client

In [7]:
from nlpsandboxclient import client

In [8]:
data_node_host = "http://data-node:8080/api/v1"
annotator_host = "http://nginx:80/api/v1"

## Load the dataset

In [9]:
datasets = client.list_datasets(host=data_node_host)

for dataset in datasets:
      print(dataset)

{'name': 'datasets/awesome-dataset'}
{'name': 'datasets/test-dataset'}
{'name': 'datasets/i2b2-test-dataset'}


## Test annotator 

In [10]:
# an example of using the annotator to annotate one note
example_note = {
   "identifier": "note-1",
   "type": "loinc:LP29684-5",
   "patientId": "507f1f77bcf86cd799439011",
   "text": "On 12/26/2020, Ms. Chloe Price met with Dr. Prescott in Seattle."
}

annotations = client.annotate_note(host=annotator_host,
                            note=example_note,
                            tool_type="nlpsandbox:date-annotator")

print(annotations)

{'textDateAnnotations': [{'start': 3, 'length': 10, 'text': '12/26/2020', 'confidence': 95.5, 'dateFormat': 'MM/DD/YYYY'}, {'start': 9, 'length': 4, 'text': '2020', 'confidence': 95.5, 'dateFormat': 'YYYY'}]}


In [14]:
# an example of using the annotator to annotate all notes in the test dataset
from nlpsandboxclient import client
notes = client.list_notes(host=data_node_host,
                  dataset_id="i2b2-test-dataset",
                  fhir_store_id="evaluation")

test_notes = list(notes)
print(len(test_notes))
annotation_store_name = 'sparknlp'
dataset_name = 'i2b2-test-dataset'
annotation_list = []
converted_annotation_list = []
for i in test_notes:
    note_name= i.pop('note_name', None)
    note_id = note_name[-6:]
    # There is an error in annotation generated from  client.annotate_note, the note_name is "dataset/.."  but it should be "datasets/.."
    note_name = "datasets"+note_name[7:]
    annotations = client.annotate_note(host=annotator_host,
                            note=i,
                            tool_type="nlpsandbox:date-annotator")
    #annotation_name = f"datasets/{dataset_name}/annotationStores/{annotation_store_name}/annotations/{note_id}"
    converted_annotation = {"annotationSource":{"resourceSource":{"name":note_name}},"textContactAnnotations": [], "textCovidSymptomAnnotations":[],"textDateAnnotations":annotations["textDateAnnotations"],"textIdAnnotations":[],"textLocationAnnotations": [],"textPersonNameAnnotations": []}
    converted_annotation_list.append(converted_annotation)

514


In [13]:
converted_annotation_list

[{'annotationSource': {'resourceSource': {'name': 'datasets/i2b2-test-dataset/fhirStores/evaluation/fhir/Note/110-01'}},
  'textContactAnnotations': [],
  'textCovidSymptomAnnotations': [],
  'textDateAnnotations': [{'start': 16,
    'length': 4,
    'text': '2069',
    'confidence': 95.5,
    'dateFormat': 'YYYY'},
   {'start': 89,
    'length': 8,
    'text': 'November',
    'confidence': 95.5,
    'dateFormat': 'MMMM'}],
  'textIdAnnotations': [],
  'textLocationAnnotations': [],
  'textPersonNameAnnotations': []},
 {'annotationSource': {'resourceSource': {'name': 'datasets/i2b2-test-dataset/fhirStores/evaluation/fhir/Note/110-02'}},
  'textContactAnnotations': [],
  'textCovidSymptomAnnotations': [],
  'textDateAnnotations': [{'start': 16,
    'length': 4,
    'text': '2075',
    'confidence': 95.5,
    'dateFormat': 'YYYY'},
   {'start': 70,
    'length': 4,
    'text': '8249',
    'confidence': 95.5,
    'dateFormat': 'YYYY'}],
  'textIdAnnotations': [],
  'textLocationAnnotation

In [15]:
## store the annotation to the data node
client.store_annotations(host=data_node_host,
                 dataset_id=dataset_name,
                 annotation_store_id= annotation_store_name,
                 annotations = converted_annotation_list,
                 delete_existing_annotations = True)

Deleted existing Annotation Store
Created Annotation Store
